# --- Day 7: No Space Left On Device ---

https://adventofcode.com/2022/day/7

In [1]:
from collections import defaultdict

## Get Input Data

In [2]:
def get_data(filename):
    """Get data for day 7 puzzle.
    
    Need to build a tree for the filesystem.
    
    Parameters
    ----------
    filename : str
        Name of input file containing all the filesystem commands and results.

    Returns
    -------
    fs : defaultdict(int)
        Each key is a full path in the filesystem, and its value is the total size of the files
        and subdirectories at that path.
    """
    fs = defaultdict(int)
    current_path = '/'

    with open(f'../inputs/{filename}.txt') as f:
        for line in f:
            line = line.rstrip()
            if line[0].isdigit():
                size, _ = line.split()  # No need to keep track of filenames...
                fs[current_path] += int(size)
            elif line.startswith('dir'):
                pass  # Nothing to do here...
            elif line.startswith('$ cd'):
                if line.endswith('/'):  # Country roads... take me home...
                    current_path = '/'
                elif line.endswith('..'):  # Movin' on up... to a delux apartment in the sky
                    # When we are back-tracking, we need to update the current_path AND 
                    # add in the size of the previous_path we're leaving
                    prev_path = current_path
                    last_subdirectory_index = current_path.rfind('/', 0, -1)
                    current_path = current_path[:last_subdirectory_index+1]
                    fs[current_path] += fs[prev_path]
                else:  # Mov into a subdirectory and then update the current_path string
                    _, _, _dir = line.split()
                    current_path += f'{_dir}/'

    # Need to add the last directory size to each of its parents, all the way back up to the root
    for _ in range(current_path.count('/')-1):
        last_subdirectory_index = current_path.rfind('/', 0, -1)
        prev_path = current_path
        current_path = current_path[:last_subdirectory_index+1]
        fs[current_path] += fs[prev_path]

    print(current_path)  # should end back at '/'
    return fs

In [3]:
get_data('test_filesystem')

/


defaultdict(int, {'/': 48381165, '/a/': 94853, '/a/e/': 584, '/d/': 24933642})

## Part 1
---

### Run on Test Data

In [4]:
fs = get_data('test_filesystem')
sum([x for x in fs.values() if x < 100000]) == 95437

/


True

### Run on Input Data

In [5]:
fs = get_data('filesystem')
sum([x for x in fs.values() if x < 100000])

/


1989474

## Part 2
---

In [6]:
def find_smallest_dir_to_delete(filename):
    TOTAL_SPACE = 70_000_000
    REQUIRED_SPACE = 30_000_000

    fs = get_data(filename)
    used_space = fs['/']
    unused_space = TOTAL_SPACE - used_space

    needed_free_space = REQUIRED_SPACE - unused_space

    smallest = TOTAL_SPACE
    for k, v in fs.items():
        if v > needed_free_space:
            smallest = min(smallest, v)

    return smallest

### Run on Test Data

In [7]:
find_smallest_dir_to_delete('test_filesystem') == 24933642

/


True

### Run on Input Data

In [8]:
find_smallest_dir_to_delete('filesystem')

/


1111607